In [21]:
!pip install chemdataextractor 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [22]:
!cde data download

Successfully downloaded 0 new data packages (18 existing)


In [23]:
from chemdataextractor import Document
from chemdataextractor.model import Compound
from chemdataextractor.doc import Paragraph, Heading



In [24]:
d = Document(
    Heading(u'S'),
    Paragraph(u'drying')
)

In [25]:


from chemdataextractor.model import BaseModel, StringType, ListType, ModelType

class BoilingPoint(BaseModel):
    value = StringType()
    units = StringType()
    
Compound.boiling_points = ListType(ModelType(BoilingPoint))



In [26]:
import re #Llama a una libreria que se llama regex (expresiones regulares) = re 
from chemdataextractor.parse import R, I, W, Optional, merge

In [68]:
import re
from chemdataextractor.parse import R, I, W, Optional, merge

prefix = (R(u'^b\.?p\.?$', re.I) | I(u'dried')|I(u'at')).hide()
units = (W(u'°') + Optional(R(u'^[CFK]\.?$')))(u'units').add_action(merge)
value = R(u'^\d+(\.\d+)?$')(u'value')
bp = (prefix + value + units)(u'bp')

In [70]:
from chemdataextractor.parse.base import BaseParser
from chemdataextractor.utils import first

class BpParser(BaseParser):
    root = bp

    def interpret(self, result, start, end):
        compound = Compound(
            boiling_points=[
                BoilingPoint(
                    value=first(result.xpath('./value/text()')),
                    units=first(result.xpath('./units/text()'))
                )
            ]
        )
        yield compound

Paragraph.parsers = [BpParser()]

#Running the New Parser

d = Document(
    Heading(u'S'),
    Paragraph(u'dried at 80 °C, calcined at 50°C, and then reimpregnated at 100 °C')
)

d.records.serialize()



[{'boiling_points': [{'value': '80', 'units': '°C'}]},
 {'boiling_points': [{'value': '50', 'units': '°C'}]},
 {'boiling_points': [{'value': '100', 'units': '°C'}]}]

In [29]:
import re #Llama a una libreria que se llama regex (expresiones regulares) = re 
from chemdataextractor.parse import R, I, W, Optional, merge #funciones definidas dentro de chemdataextractor 
# R --> regex 
# I --> IWord : Encuentra el texto sin importar las mayúsculas y las minúsculas
# W --> W : Encuentra el texto exactamente 
# Documentación de regex: https://docs.python.org/3/library/re.html

prefix = (R(u'^b\.?p\.?$', re.I) | I(u'boiling') + I(u'point')).hide()
#(u'^b\.?p\.?$', re.I) | I(u'boiling') + I(u'point') --> Expresión regular 
# u --> unicodes 
#'^b\.?p\.?$' --> ' ' --> string, ^b --> Encuentra el inicio del string que tiene una b, \. --> encuentra un punto, 
#? --> Identifica 0 o 1 veces que ocurra la cadena de texto anterior en el texto
#p\. --> Encuentra p.
#$ --> Final del string 
# b.fasdfvdfgp. 
#re.I : Encuentra el texto sin importar las mayúsculas y las minúsculas
#No importa si es b.p. o B.P.

# | --> barra vertical identifica lo de la derecha y lo de la izquierda. 
# I --> no importa si son mayúsculas o minúsculas (u'boiling')
#.hide() --> ocultalo 

units = (W(u'°') + Optional(R(u'^[CFK]\.?$')))(u'units').add_action(merge)
value = R(u'^\d+(\.\d+)?$')(u'value')
#^ --> inicio de la cadena de texto 
#\ --> Quita lo especial de un carácter especial 
# \d --> Encuentra un número (0-9) y otros carácteres rel con matematicass
#+ --> Encuentra otra vez la expresión regular que le precede 
#() --> Encuentra la expresión regular dentro del paréntesis e indica el inicio de un grupo 

bp = (prefix + value + units)(u'bp')